# DYLE+GPT with Aveni data

In [ ]:
!pip install --upgrade openai
!pip install rouge

In [ ]:
import numpy as np
from rouge import Rouge
from pprint import pprint
import nltk
nltk.download('punkt')
from nltk import word_tokenize
import json
from sys import displayhook
from openai import OpenAI
import tiktoken

## utils

In [ ]:
client = OpenAI(api_key='') # Insert OpenAI API key

In [ ]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

#num_tokens_from_string(str, "cl100k_base")

# Following QMSum
def tokenize(sent):
    tokens = ' '.join(word_tokenize(sent.lower()))
    return tokens

# filter some noises caused by speech recognition
def clean_data(text):
    text = text.replace('{ vocalsound }', '')
    text = text.replace('{ disfmarker }', '')
    text = text.replace('a_m_i_', 'ami')
    text = text.replace('l_c_d_', 'lcd')
    text = text.replace('p_m_s', 'pms')
    text = text.replace('t_v_', 'tv')
    text = text.replace('{ pause }', '')
    text = text.replace('{ nonvocalsound }', '')
    text = text.replace('{ gap }', '')
    return text

## Data loading

In [ ]:
with open("../Data/Aveni/all/annotated_demo_08_11.json", "r") as demo: 
     demo_queries = json.load(demo)

# make lists with queries
topic_list = []
general_query_list = []
specific_query_list = []

for i in demo_queries['topic_list']:
    for key, val in i.items():
        if key == "topic":
              topic_list.append(val)
for i in demo_queries['general_query_list']:
    for key, val in i.items():
        if key == "query":
              general_query_list.append(val)
for i in demo_queries['specific_query_list']:
    for key, val in i.items():
        if key == "query":
              specific_query_list.append(val)

In [ ]:
#retrieved topk
with open('../Data/AllQMSum_AVENI_DYLE_test.txt', 'r') as file:
    text = file.read()

In [ ]:
queries_and_context = {}
queries = general_query_list + specific_query_list
dyle_answers = []

word_tok_text = word_tokenize(text)
topk = []
for i in range(len(word_tok_text)):
    this_topk = []
    this_ans = []
    if word_tok_text[i] == 'topk':     
        i = i+2
        while word_tok_text[i] != '--':
            this_topk.append(word_tok_text[i])
            i = i+1
        this_topk = ' '.join(this_topk)
        topk.append(this_topk)
        
    if word_tok_text[i] == 'Pred':
        i = i+2
        while word_tok_text[i] != '--':
            this_ans.append(word_tok_text[i])
            i = i+1
        this_ans = ' '.join(this_ans)
        dyle_answers.append(this_ans)
        
        
for i in range(len(queries)):
    queries_and_context[queries[i]] = topk[i]

## Querying the model

In [ ]:
def get_info(topk: str) -> str:
    info = f"""You're the assistant for a financial advisor. Use the below retreievd content from a meeting transcript to answer all questions. If the answer cannot be found, write "n/a"

retrieved content:
\"\"\"
{topk}
\"\"\"

"""
    return info

In [ ]:
def ask(query: str, topk: str,
    print_message: bool = False) -> str:
  #  if print_message:
   #     print(message)
    messages = [
        {"role": "system", "content": get_info(topk)},
        {"role": "user", "content": query},
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=messages,
        temperature=0.0
    )
    response_message = response.choices[0].message.content
    return response_message

In [ ]:
hyp_list = []
for question, topk in queries_and_context.items():
    response = ask(question, topk)
    hyp_list.append(tokenize(response))

## Evaluation

In [ ]:
with open('../Data/Aveni/all/annotated_demo_08_11.json') as refs:
    refs_data = json.load(refs)

ref_list = []
for i in refs_data['general_query_list']:
        for key, val in i.items():
            if key == "answer": 
                ref_list.append(tokenize(val))
for i in refs_data['specific_query_list']:
        for key, val in i.items():
            if key == "answer": 
                ref_list.append(tokenize(val))

Summary evaluation

In [ ]:


data = []
for i in range(len(hyp_list)):
    data.append({'hyp': hyp_list[i], 'ref': ref_list[i]})

hyps, refs = map(list, zip(*[[d['hyp'], d['ref']] for d in data]))
rouge = Rouge()

scores = rouge.get_scores(hyps, refs)
# or
avg_scores = rouge.get_scores(hyps, refs, avg=True)

pprint(avg_scores)

In [ ]:
with open('dyle_gpt_output.txt', 'w') as f:
    for line in data:
        f.write(f"{line}\n")
f.close()

In [ ]:
print(data[0])

{'hyp': 'the main reason for seeking advice is because the individual is not an expert in financial planning , particularly when it comes to pensions . they have a large workplace pension and want to ensure they will have enough money for retirement . they were referred to acme limited by a friend who spoke highly of the company .', 'ref': 'the client is seeking advice because of his lack of expertise in financial planning . he wants to ensure that he will have enough money for retirement , and he chose acme limited because his friend carol recommended the company . the client values the peace of mind that comes from consulting with a knowlegeable advisor .'}


Token number

In [ ]:
pprint(queries_and_context[queries[10]])

('hi . sorry , sorry , the paceman . and when is retirement for you ? no , '
 "just me on my own . okay , great . so the questions have n't changed , so "
 'hopefully it all comes flooding back to you , mr. smith . um , so just to '
 'remind you , try to get your investment portfolio out of your head . okay . '
 'and just try to think about how you personally feel at this moment in time '
 'between risk and reward . okay . so question one , how would you describe '
 'your typical attitude , uh , and making important financial decisions ? '
 "would you say you 're very adventurous , fairly adventurous , balance of "
 'adventure and caution ? fairly careful or very careful ? yeah , that sounds '
 "good . okay , cool . sure , that sounds good . that 's it . yeah , that , "
 'that sounds like a good summary of , of what i was trying to say , okay . '
 'yeah , sounds good . yes , so i inherited that property from my late father '
 '. good , good way to view life really . um , and how would 

In [ ]:

retrieved_tok = []
gen_tok = []
i = 0
for question, topk in queries_and_context.items():
    retrieved_tok.append(num_tokens_from_string(topk, "cl100k_base"))
    gen_tok.append(num_tokens_from_string(hyp_list[i], "cl100k_base"))
    i += 1
print('average retrieved tokens DYLE: ', np.mean(retrieved_tok))
print('average generated tokens GPT: ', np.mean(gen_tok))


average retrieved tokens DYLE:  814.45
average generated tokens GPT:  63.625


In [ ]:
#average generated tokens DYLE
print('average generated tokens DYLE: ', np.mean([num_tokens_from_string(i, "cl100k_base") for i in dyle_answers]))


average generated tokens DYLE:  110.825
